## This notebook shows all the means to index the diagrams in DiagramsDataset

In [ ]:
import pathlib
import shutil
import tempfile
import time

from laueimproc import Diagram, DiagramsDataset
from laueimproc.io import get_samples  # gives access to the dataset

### Link the diagram instance to the index.
* If the index is contained in the diagram file name, so skip this step, the diagrams will be indexed automatically.

In [ ]:
# it is possible to overwrite laueimproc.classes.dataset.default_diag2ind

def stupid_diag2ind(diagram: Diagram) -> int:
    """Take a diagram and return the index."""
    return int(diagram.file.name[-6:-4])

### Set the geometry of the grid.
* This step is optional, but allows you to acces the element from the real sample position

In [ ]:
# 1 : define the rule that maps a diagram index its position in 2 dimensions

def position_func(index: int) -> tuple[float, float]:  # can return tuple[int, int]
    """Return the position x, y of the diagram in micron."""
    y_ind, x_ind = divmod(index, 20)  # assume grid shape is 20 x 5 samples
    x_micron = 20.0 * x_ind  # assume x step is 20 microns (fast index)
    y_micron = 30.0 * y_ind  # assume y step is 30 microns (slow index)
    return x_micron, y_micron  # must to return 2 numbers

# 2 : simply provide the argument position=position_func so that the DiagramsDataset integrates the positions

### Init a diagram, see the `api_init_dataset` notebook for more informations.

In [ ]:
dataset = DiagramsDataset(get_samples().glob("*.jp2"), position=position_func)
print(dataset)

### Access to one diagram.

In [ ]:
# from the diagram index
print(f"from index  0: {repr(dataset[0])}")
print(f"from index  1: {repr(dataset[1])}")
print(f"from index -1: {repr(dataset[-1])}")
print(f"from index -2: {repr(dataset[-2])}")
try:
    dataset[100]
except IndexError as err:
    print(err)

# # from the grid position (argument grid has to be provided)
# print(f"from coord (  0,   0): {repr(dataset[0, 0])}")
# print(f"from coord (200,  60): {repr(dataset[200, 60])}")
# print(f"from coord (380, 120): {repr(dataset[380, 120])}")

### Access a subset of diagrams.

In [ ]:
# from, the diagrams indices
print(set(dataset[1:50:10]))  # slice
print(set(dataset[[0, 10, 11, 12]]))  # indices
print(set(dataset[[(i % 20) == 0 for i in range(len(dataset))]]))  # condition

# diagrams are shared
print()
print(dataset[0])
sub_dataset = dataset[:1]
print(sub_dataset[0])  # exactly the same as previous
sub_dataset[0].find_spots()
print(dataset[0])